In [3]:
import pandas as pd

#### Ucitavanje dataseta

In [4]:
df = pd.read_csv('../datasets/fake-news/balansed_fake_news_dataset.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49550 entries, 0 to 49549
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    49550 non-null  object
 1   text     49548 non-null  object
 2   subject  49550 non-null  object
 3   date     34190 non-null  object
 4   fake     49550 non-null  bool  
dtypes: bool(1), object(4)
memory usage: 1.6+ MB


### Ciscenje teksta

Izbacivanje reci uz pomoc regexa i stopwordsa i ne utice previse na tf idf zbog idf dela algoritma, **ali** empirijski smo dosli do zakljucka da ipak moze podici preciznost za par procenata, pa cemo provuci kroz istu funkciju kroz koju provlacimo i za ostale eksperimente.

In [5]:
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')
stopwords = stopwords.words('english')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/lukakrickovic/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [11]:
import re

df = df.reset_index(drop=True)

def clean_text(text: str) -> str:
    REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
    BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
    STOPWORDS = set(stopwords)
    text = text.lower()
    text = REPLACE_BY_SPACE_RE.sub(' ', text)
    text = BAD_SYMBOLS_RE.sub('', text)
    text = ' '.join(word for word in text.split() if word not in STOPWORDS)
    return text

df['text'].dropna()
df = df.loc[df.text.apply(type) != float]
df['text'] = df['text'].apply(clean_text)
df['text'] = df['text'].str.replace('\d+', '')

/var/folders/ny/km1m35px577gl9cs19xd2scm0000gn/T/ipykernel_16749/1871003461.py:18: FutureWarning: The default value of regex will change from True to False in a future version.
  df['text'] = df['text'].str.replace('\d+', '')


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 49548 entries, 0 to 49547
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    49548 non-null  object
 1   text     49548 non-null  object
 2   subject  49548 non-null  object
 3   date     34190 non-null  object
 4   fake     49548 non-null  bool  
dtypes: bool(1), object(4)
memory usage: 1.9+ MB


Vidimo da je bilo 2 float reda!

#### TF-IDF

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
analyze = vectorizer.build_analyzer()

Treniramo i dobijamo vektorizovan tekst

Belezimo vreme jer je relevantno za poredjenje sa drugim metodama

In [20]:
%%time
X = vectorizer.fit_transform(df.text)

CPU times: user 8.09 s, sys: 196 ms, total: 8.29 s
Wall time: 8.29 s


In [21]:
Y = pd.get_dummies(df['fake']).values

In [22]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state = 0)

Priprema klasifikatora

In [23]:
from sklearn.naive_bayes import MultinomialNB

classifier_nb = MultinomialNB()
classifier_nb.fit(X_train, y_train[:, 0])

MultinomialNB()

In [24]:
y_pred = classifier_nb.predict(X_test)

#### Rezultati

In [25]:
from sklearn.metrics import accuracy_score, classification_report

classification_report_first = classification_report(y_test[:, 0], y_pred)

print('\n Preciznost: ', accuracy_score(y_test[:, 0], y_pred))
print('\nIzvestaj klasifikacije:')
print('======================================================')
print('\n', classification_report_first)


 Preciznost:  0.8688193743693239

Izvestaj klasifikacije:

               precision    recall  f1-score   support

           0       0.94      0.79      0.86      4976
           1       0.82      0.95      0.88      4934

    accuracy                           0.87      9910
   macro avg       0.88      0.87      0.87      9910
weighted avg       0.88      0.87      0.87      9910

